In [ ]:
import pickle
import json
import re
import os
import math
from tqdm import tqdm

1.读取初始数据

In [ ]:
data = []
with open('baike_triples.txt', 'r', encoding='utf8') as f:
    for line in tqdm(f):
        data.append(line.strip().split('\t'))
print(len(data), data[0])

2.保留元素全为中文的三元组

In [ ]:
all_chinese = re.compile('^[\u4e00-\u9fa5]*$')
new_data = []
for triple in tqdm(data):
    if bool(re.search(all_chinese, triple[0])) and bool(re.search(all_chinese, triple[1])) and bool(re.search(all_chinese, triple[2])):
        new_data.append(triple)
len(new_data)

3.实体字典

In [ ]:
entities = {}
for triple in tqdm(new_data):
    entities[triple[0]] = entities.get(triple[0], {})
    entities[triple[0]][triple[1]] = triple[2]

In [ ]:
if not os.path.isdir('processed'):
    os.mkdir('processed')
with open('processed/entities.pkl', 'wb') as f:
    pickle.dump(entities, f)

In [ ]:
e_set = set()
with open('processed/entity.txt', 'w', encoding='utf8') as f:
    for e in tqdm(entities.keys()):
        if e not in e_set:
            e_set.add(e)
            f.write(e+'\n')
len(e_set)

4.获取句子集合

In [ ]:
def process_sentence(sent):
    sent = re.sub(r' +', ' ', sent)
    sent = re.sub(r'[^\u4e00-\u9fa5,\?\!，。？：:！、；\(\)（） ]', '', sent)
    sent = re.split(r'[\?\!。？！]', sent)
    return sent

In [ ]:
sents = []
for triple in tqdm(data):
    if triple[1]=='BaiduCARD':
        sent = process_sentence(triple[2])
        if sent != "":
            sents.append(sent)
print(len(sents), sents[0])

5.句子存为100个文件

In [ ]:
if not os.path.isdir('processed/sentences'):
    os.mkdir('processed/sentences')

    begin = 0
count = math.ceil(len(sents)*1.0/100)
end = min(begin+count, len(sents))
for i in tqdm(range(100)):
    with open('processed/sentences/sen'+str(i), 'wb') as f:
        pickle.dump(sents[begin:end], f)
    begin = end
    end = min(begin+count, len(sents))